In [ ]:
import pandas as pd
df = pd.read_csv('../input/bengaluru-house-price-data/Bengaluru_House_Data.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df['area_type'].unique()

In [ ]:
df['area_type'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
#area_type,availability,society,balcony
df1 = df.drop(['area_type','availability','society','balcony'],axis=1)

In [ ]:
df1.head()

In [ ]:
df1.dtypes

In [ ]:
loc_count = df1['location'].value_counts()

In [ ]:
loc_count[loc_count < 10]

In [ ]:
df1.isnull().sum()

In [ ]:
df1['size'] = df1['size'].fillna('0')

In [ ]:
k = '2 BHK'
k.split(' ')[0]

In [ ]:
def bedroom(x):
    return int(x.split(' ')[0])

In [ ]:
bedroom('2 BHK')

In [ ]:
df1['size'] = df1['size'].apply(bedroom)

In [ ]:
df1.head()

In [ ]:
df1.dtypes

In [ ]:
df1['total_sqft'].astype(float)

In [ ]:
def is_num(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
is_num('2100 - 2850')

In [ ]:
df1[df1['total_sqft'].apply(is_num)]

In [ ]:
df1[~df1['total_sqft'].apply(is_num)]

In [ ]:
v = '2830 - 2882'
(float(v.split('-')[0]) + float(v.split('-')[1]))/2

In [ ]:
def convert_to_num(v):
    val = v.split('-')
    if (len(val)) == 2:
        return ((float(val[0]) + float(val[1]))/2)
    try:
        return float(v)
    except:
        return None

In [ ]:
df2 = df1.copy()

In [ ]:
df2['total_sqft'] = df2['total_sqft'].apply(convert_to_num)

In [ ]:
df2.dtypes

In [ ]:
df2.isnull().sum()

In [ ]:
df2 = df2.dropna()

In [ ]:
df2.isnull().sum()

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
#'Bipul'
#' Bipul'
s = ' Bipul'
s.strip()

In [ ]:
def remove_space(x):
    return x.strip()

In [ ]:
remove_space(' Bipul')

In [ ]:
df2['location'] = df2['location'].apply(remove_space)

In [ ]:
loc_count = df2['location'].value_counts()
loc_count

In [ ]:
(loc_count > 10).sum()

In [ ]:
(loc_count < 10).sum()

In [ ]:
loc_count_less_10 = loc_count[loc_count <= 10]

In [ ]:
'Gulakamale' in loc_count_less_10

In [ ]:
'Whitefield' in loc_count_less_10

In [ ]:
def replace_less_10(x):
    if x in loc_count_less_10:
        return 'Other'
    else:
        return x

In [ ]:
replace_less_10('Gulakamale')

In [ ]:
df2['location'] = df2['location'].apply(replace_less_10)

In [ ]:
len(df2['location'].unique())

In [ ]:
df2.head()

#Outliers

In [ ]:
df2['size'].unique()

In [ ]:
df2['size'].value_counts()

In [ ]:
df2.shape

In [ ]:
df3 = df2[~(df2['total_sqft'] / df2['size'] < 300)]

In [ ]:
df3.head()

In [ ]:
df3.shape

In [ ]:
df3['PPS'] = (df3['price'] * 100000) / df3['total_sqft']

In [ ]:
df3.head()

In [ ]:
df3['PPS'].describe()

In [ ]:
df3.groupby('location')['PPS'].mean().sort_values()

In [ ]:
df3.groupby('location')['PPS'].max().sort_values()

In [ ]:
df3.groupby('location')['PPS'].min().sort_values()

In [ ]:
for key,i in df3.groupby('location'):
    print(key)

In [ ]:
df4 = pd.DataFrame()
import numpy as np
for key,dfloc in df3.groupby('location'):
    #print(key,dfloc)
    m = np.mean(dfloc['PPS'])
    st = np.std(dfloc['PPS'])
    #print(key,m,st)
    
    extracted_df = dfloc[(dfloc['PPS'] > (m-st)) & (dfloc['PPS'] < (m+st))]
    df4 = pd.concat([df4,extracted_df],ignore_index=True)

In [ ]:
df4.head()

In [ ]:
df4.groupby('location')['PPS'].max().sort_values()

In [ ]:
df4.groupby('location')['PPS'].min().sort_values()

In [ ]:
df4.head()

In [ ]:
df4['bath'].unique()

In [ ]:
df4['size'].unique()

In [ ]:
import seaborn as sns
sns.countplot(df4['bath'])

In [ ]:
df4[df4['bath'] > 10]

In [ ]:
df4[df4['bath'] > df4['size'] + 2]

In [ ]:
df5 = df4[df4['bath'] < df4['size'] + 2]

In [ ]:
df5.shape

In [ ]:
df5.head()

In [ ]:
df6 = df5[['location','size','total_sqft','bath','price']]

In [ ]:
df6.head()

In [ ]:
df_encoded = pd.get_dummies(df6['location'])

In [ ]:
df7 = pd.concat([df6,df_encoded],axis='columns')

In [ ]:
df7.head()

In [ ]:
df8 = df7.drop('location',axis=1)

In [ ]:
df8.head()

In [ ]:
df8.dtypes

In [ ]:
X = df8.drop('price',axis=1)
Y = df8[['price']]

In [ ]:
X

In [ ]:
df8.to_csv('banglore_final.csv')

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y)

In [ ]:
from sklearn.linear_model import LinearRegression
lmodel = LinearRegression()

lmodel.fit(xtrain,ytrain)

In [ ]:
#Error values
from sklearn.metrics import mean_absolute_error,mean_squared_error
print('Training Error',mean_absolute_error(ytrain,lmodel.predict(xtrain)))
print('Testing Error',mean_absolute_error(ytest,lmodel.predict(xtest)))

In [ ]:
loc = 'Vijayanagar'
np.where(X.columns == loc)[0][0]

In [ ]:
#size,tota sqft, bath,loc
#3,2000,3,'Vijayanagar'
loc = np.where(X.columns == 'Vijayanagar')[0][0]
loc

In [ ]:
np.zeros(len(X.columns))

In [ ]:
def predict_my_price(location,beds,sqft,baths):
    loc = np.where(X.columns == location)[0][0]
    
    x = np.zeros(len(X.columns))
    x[0] = beds
    x[1] = sqft
    x[2] = baths
    
    x[loc] = 1
    
    return lmodel.predict([x])

In [ ]:
X.columns

In [ ]:
price = predict_my_price('2nd Phase Judicial Layout',3,2000,2)
print(price)

In [ ]:
import pickle
with open('banglore_real_estate.pickle','wb') as f:
    pickle.dump(lmodel,f)

In [ ]:
import json
columns = { 'data_columns' : [col.lower() for col in X.columns] } 
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

# ****Solving Problem using Polynomial Features****

In [ ]:
lmodel.coef_

In [ ]:
xtrain

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
pol = PolynomialFeatures(2)
pol.fit(xtrain)

In [ ]:
xtrain_pol = pol.transform(xtrain)

In [ ]:
print(xtrain.shape)
print(xtrain_pol.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
pmodel = LinearRegression()
pmodel.fit(xtrain_pol,ytrain)

In [ ]:
ytrain_pred = pmodel.predict(xtrain_pol)

In [ ]:
xtest_pol = pol.transform(xtest)
ytest_pred = pmodel.predict(xtest_pol)

In [ ]:
from sklearn.metrics import mean_absolute_error
print('Training Error',mean_absolute_error(ytrain,ytrain_pred))
print('Testing Error',mean_absolute_error(ytest,ytest_pred))